In [216]:
import pandas as pd
import numpy as np
from math import *

from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV, ParameterGrid

import matplotlib.pyplot as plt

from keras.models import Sequential 
from keras.layers import Dense 
from keras.layers import Flatten 
from keras.layers.convolutional import Conv1D 
from keras.layers.convolutional import MaxPooling1D 
from keras.layers import LSTM

from sklearn.metrics import mean_squared_error

### 01. Function 

In [210]:
# transform data to supervised learning problem
def transform_supervised(df, number=5):
    i=0
    X,y=list(), list()
    while i<= (len(df) - number ):
        X.append(df[i:i+number].values)
        y.append(df[i+number:i+number+1].values)
        i=i+1
    return X,y
    
def data_supervised(df, n_in=3, n_out=1, dropna =True):
    cols,names =list(), list()
    # number of lag
    n_vars = df.shape[1]
    for i in range( n_in, 0, -1):
        cols.append(df.shift(-i))
        names+=['var%d(t - %d)' %(j+1, i) for j in range(n_vars)]
    for i in range(0, n_out):
        cols.append(df.shift(i))
        if i==0:
            names+=['var%d(t)' %(j+1) for j in range(n_vars)]
        else:
            names+=['var%d(t + %d)' %(j+1, i) for j in range(n_vars)]
    #concat all dataset
    data=pd.concat(cols, axis=1)
    data.columns=names
    if dropna:
        data.dropna(inplace=True)
    return data


#define the model for MLP
def MLP_model(input_dim, optimizer = 'adam', loss='mse'):
    model = Sequential() 
    model.add(Dense(100, activation='tanh', input_dim=3))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(1)) 
    model.compile(optimizer=optimizer, loss=loss) 
    return model 
# CNN model
def CNN_model():
    model = Sequential() 
    model.add(Conv1D(64, 2, activation='relu', input_shape=(3, 1))) 
    model.add(MaxPooling1D()) 
    model.add(Flatten()) 
    model.add(Dense(100, activation='tanh'))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(1)) 
    model.compile(optimizer='adam', loss='mse')
    return model 




In [224]:
# transform data to dataframe

fh = open('')

num_list = []

read_lines = fh.readlines()
for line in read_lines:
    num_list.append(line)

fh.close()
data = num_list[0].split(' ')
Date=list()
Births= list()
for num in data:
    Date.append(num.split(',')[0][1:-1])
    Births.append(num.split(',')[1])
Date=Date[1:]
Births=Births[1:]

total_daily=pd.DataFrame({'Date':Date, 'Births': Births})
total_daily['Date']=pd.to_datetime(total_daily.Date, format='%Y-%m-%d')
total_daily.index=total_daily.Date
total_daily=total_daily.drop(['Date'], axis=1)
total_daily.to_csv(' ')

FileNotFoundError: [Errno 2] No such file or directory: ''

### 02 How to transform data to time series

In [87]:
data_supervised(total_daily, n_in=3, n_out=2, dropna =True).head(4)

,var1(t - 3),var1(t - 2),var1(t - 1),var1(t),var1(t + 1)
Date,,,,,
1959-01-02,44,31,30,32,35
1959-01-03,29,44,31,30,32
1959-01-04,45,29,44,31,30
1959-01-05,43,45,29,44,31


### 03. MLP for time series forecasting

In [92]:
data=data_supervised(total_daily, n_in=3, n_out=1, dropna =True)
data.head(4)

,var1(t - 3),var1(t - 2),var1(t - 1),var1(t)
Date,,,,
1959-01-01,31,30,32,35
1959-01-02,44,31,30,32
1959-01-03,29,44,31,30
1959-01-04,45,29,44,31


In [151]:
#define the dataset
n_train=int(len(data)*0.8)
X_train=data.iloc[:,0:-1][0:n_train]
X_test=data.iloc[:,0:-1][n_train:]
y_train=data.iloc[:,[-1]][0:n_train]
y_test = data.iloc[:,[-1]][n_train:]
print(y_test)
print(X_test)

           var1(t)
Date              
1959-10-17      49
1959-10-18      45
1959-10-19      43
1959-10-20      42
1959-10-21      38
...            ...
1959-12-24      38
1959-12-25      44
1959-12-26      34
1959-12-27      37
1959-12-28      52

[73 rows x 1 columns]
           var1(t - 3) var1(t - 2) var1(t - 1)
Date                                          
1959-10-17          42          43          45
1959-10-18          38          42          43
1959-10-19          47          38          42
1959-10-20          38          47          38
1959-10-21          36          38          47
...                ...         ...         ...
1959-12-24          37          34          44
1959-12-25          52          37          34
1959-12-26          48          52          37
1959-12-27          55          48          52
1959-12-28        50\n          55          48

[73 rows x 3 columns]


In [187]:
model=MLP_model(3)
#model.fit(X_train, y_train, epochs=200, verbose=1)

In [160]:
#predict X_test with the train model
y_pred = model.predict(X_test)
error=np.sqrt(mean_squared_error(y_pred, y_test))
error

5.953957928825948

### 04. CNN for time series 

In [171]:
X_train.values.reshape(X_train.shape[0], X_train.shape[1],1)

array([['44'],
       ['31'],
       ['30']], dtype=object)

In [196]:
n_train=int(len(data)*0.8)
X_train=data.iloc[:,0:-1][0:n_train].values.reshape(X_train.shape[0], X_train.shape[1],1)
X_test=data.iloc[:,0:-1][n_train:].values.reshape(X_test.shape[0], X_test.shape[1],1)
y_train=data.iloc[:,[-1]][0:n_train]
y_test = data.iloc[:,[-1]][n_train:]
#print(y_test)
#print(X_test)

In [212]:


CNN_model=CNN_model()
CNN_model.fit(X_train, y_train, epochs=200, verbose=1)

Epoch 1/200
289/289 [==============================] - 0s 1ms/step - loss: 1376.6096
Epoch 2/200
289/289 [==============================] - 0s 62us/step - loss: 890.4478
Epoch 3/200
289/289 [==============================] - 0s 76us/step - loss: 526.9384
Epoch 4/200
289/289 [==============================] - 0s 76us/step - loss: 255.5103
Epoch 5/200
289/289 [==============================] - 0s 73us/step - loss: 109.8710
Epoch 6/200
289/289 [==============================] - 0s 72us/step - loss: 59.9245
Epoch 7/200
289/289 [==============================] - 0s 79us/step - loss: 57.7810
Epoch 8/200
289/289 [==============================] - 0s 72us/step - loss: 58.2984
Epoch 9/200
289/289 [==============================] - 0s 72us/step - loss: 57.8299
Epoch 10/200
289/289 [==============================] - 0s 76us/step - loss: 57.4001
Epoch 11/200
289/289 [==============================] - 0s 79us/step - loss: 57.7241
Epoch 12/200
289/289 [==============================] - 0s 83us/step 

289/289 [==============================] - 0s 110us/step - loss: 56.5525
Epoch 97/200
289/289 [==============================] - 0s 124us/step - loss: 57.2835
Epoch 98/200
289/289 [==============================] - 0s 121us/step - loss: 57.3316
Epoch 99/200
289/289 [==============================] - 0s 162us/step - loss: 56.8040
Epoch 100/200
289/289 [==============================] - 0s 168us/step - loss: 56.4965
Epoch 101/200
289/289 [==============================] - 0s 189us/step - loss: 56.3185
Epoch 102/200
289/289 [==============================] - 0s 169us/step - loss: 56.3809
Epoch 103/200
289/289 [==============================] - 0s 184us/step - loss: 55.7769
Epoch 104/200
289/289 [==============================] - 0s 177us/step - loss: 55.7613
Epoch 105/200
289/289 [==============================] - 0s 167us/step - loss: 56.1691
Epoch 106/200
289/289 [==============================] - 0s 184us/step - loss: 55.2780
Epoch 107/200
289/289 [==============================] - 0s 

289/289 [==============================] - 0s 97us/step - loss: 60.0276
Epoch 191/200
289/289 [==============================] - 0s 76us/step - loss: 57.1398
Epoch 192/200
289/289 [==============================] - 0s 93us/step - loss: 58.1243
Epoch 193/200
289/289 [==============================] - 0s 97us/step - loss: 55.4535
Epoch 194/200
289/289 [==============================] - 0s 100us/step - loss: 56.7413
Epoch 195/200
289/289 [==============================] - 0s 86us/step - loss: 55.2249
Epoch 196/200
289/289 [==============================] - 0s 72us/step - loss: 55.8518
Epoch 197/200
289/289 [==============================] - 0s 83us/step - loss: 56.1212
Epoch 198/200
289/289 [==============================] - 0s 110us/step - loss: 55.7195
Epoch 199/200
289/289 [==============================] - 0s 99us/step - loss: 54.8181
Epoch 200/200
289/289 [==============================] - 0s 96us/step - loss: 55.5490


In [213]:
y_pred= CNN_model.predict(X_test)
error=np.sqrt(mean_squared_error(y_pred, y_test))
error

6.190013784519519

### 05. LSTM for time series forecasting

In [222]:
model = Sequential() 
model.add(LSTM(100, activation='relu', input_shape=(3, 1))) 
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse') 
# fit model 
model.fit(X_train, y_train, epochs=100, verbose=1)

Epoch 1/100
289/289 [==============================] - 1s 4ms/step - loss: 1611.1212
Epoch 2/100
289/289 [==============================] - 0s 269us/step - loss: 1198.0863
Epoch 3/100
289/289 [==============================] - 0s 273us/step - loss: 384.3250
Epoch 4/100
289/289 [==============================] - 0s 280us/step - loss: 166.3086
Epoch 5/100
289/289 [==============================] - 0s 269us/step - loss: 85.1133
Epoch 6/100
289/289 [==============================] - 0s 293us/step - loss: 88.6333
Epoch 7/100
289/289 [==============================] - 0s 338us/step - loss: 75.5509
Epoch 8/100
289/289 [==============================] - 0s 286us/step - loss: 74.2405
Epoch 9/100
289/289 [==============================] - 0s 290us/step - loss: 71.7412
Epoch 10/100
289/289 [==============================] - 0s 297us/step - loss: 68.8704
Epoch 11/100
289/289 [==============================] - 0s 307us/step - loss: 73.3427
Epoch 12/100
289/289 [==============================] - 0s 

289/289 [==============================] - 0s 269us/step - loss: 57.6489
Epoch 97/100
289/289 [==============================] - 0s 252us/step - loss: 57.3779
Epoch 98/100
289/289 [==============================] - 0s 300us/step - loss: 55.7050
Epoch 99/100
289/289 [==============================] - 0s 273us/step - loss: 54.3921
Epoch 100/100
289/289 [==============================] - 0s 269us/step - loss: 56.1045


In [225]:
y_pred= model.predict(X_test)
error=np.sqrt(mean_squared_error(y_pred, y_test))
error

6.025848815418112